<a href="https://colab.research.google.com/github/dynamodenis/Artificial_Inteligence/blob/main/Fine_Tuning_LLMs_with_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning LLMs with Hugging Face

## Step 1: Installing and importing the libraries

In [11]:
!pip uninstall accelerate peft bitsandbytes transformers trl -y
!pip install accelerate peft==0.13.2 bitsandbytes transformers trl==0.12.0

Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
Found existing installation: peft 0.13.2
Uninstalling peft-0.13.2:
  Successfully uninstalled peft-0.13.2
Found existing installation: bitsandbytes 0.48.1
Uninstalling bitsandbytes-0.48.1:
  Successfully uninstalled bitsandbytes-0.48.1
Found existing installation: transformers 4.57.0
Uninstalling transformers-4.57.0:
  Successfully uninstalled transformers-4.57.0
Found existing installation: trl 0.12.0
Uninstalling trl-0.12.0:
  Successfully uninstalled trl-0.12.0
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.13.2-py3-none-any.whl.metadata (13 kB)
  Using cached bitsandbytes-0.48.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached transformers-4.57.0-py3-none-any.whl.metadata (41 kB)
  Using cached trl-0.12.0-py3-none-any.whl.metadata (10 kB)
Using cached peft-0.13.2-py3-none-any.whl (320 kB)
Using cached

In [ ]:
!pip install huggingface_hub

In [ ]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

## Step 2: Loading the model

In [ ]:
# Use Llama pretrained model
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="aboonaji/llama2finetune-v2",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                                                          bnb_4bit_compute_dtype= getattr(torch, "float16"),
                                                                          bnb_4bit_quant_type="nf4"))

llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

## Step 3: Loading the tokenizer

In [ ]:
llama_tockenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path="aboonaji/llama2finetune-v2",
                                                 trust_remote_code=True)
llama_tockenizer.pad_token = llama_tockenizer.eos_token
llama_tockenizer.padding_side = "right"

## Step 4: Setting the training arguments

In [ ]:
# Define your desired repository name on the Hub

REPO_ID = "dynamo-denis-llama2finetune-medical"
training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size=1, max_steps = 100, hub_model_id = REPO_ID )

In [ ]:
llama_sft_trainer = SFTTrainer(model = llama_model,
                               args = training_arguments,
                               train_dataset = load_dataset(path = "aboonaji/wiki_medical_terms_llam2_format", split="train"),
                               tokenizer = llama_tockenizer,
                               peft_config = LoraConfig(task_type="CAUSAL_LM", r=64, lora_alpha=16, lora_dropout = 0.1),
                               dataset_text_field="text"
                               )

## Step 6: Training the model

In [ ]:
# Warning: To run the training you first need to sign up here: https://wandb.ai/authorize?ref=models
# And then you will directly find in the main page your API key which you will enter in the output below.
llama_sft_trainer.train()




## Step 7: Chatting with the model

In [ ]:
user_prompt = "Please tell me more about the signs and symptoms of Cancer"
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer=llama_tockenizer, max_length=300)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]["generated_text"])

## Step 8: Push the method to hugging face

In [ ]:
from huggingface_hub import login
# Logging to hugging face
login()

# llama_model.push_to_hub(repo_name)
# llama_tockenizer.push_to_hub(repo_name)

llama_sft_trainer.push_to_hub(REPO_ID)